# Image Quality Control - Manual and Automatic Annotations Demo

This notebook demonstrates how to use the image quality control utilities for both manual and automatic annotations. Each cell corresponds to a block in the `qc_annotation_demo.py` file.

You can run each cell independently or all together to see the complete workflow.

In [3]:
# Setup - Import required modules
import sys
import os
import pandas as pd
from pathlib import Path

# Add the current directory to path for imports
current_dir = Path.cwd()
sys.path.insert(0, str(current_dir))

from utils.image_quality_qc_utils import (
    load_qc_data, save_qc_data, flag_qc, remove_qc, 
    get_qc_summary, get_flagged_images, get_unflagged_images,
    get_images_by_flag, get_images_by_annotator, QC_FLAGS,
    manual_qc, auto_qc
)

print("Successfully imported QC utilities!")
print(f"Available QC flags: {list(QC_FLAGS.keys())}")

ModuleNotFoundError: No module named 'utils.mask_utils'

In [2]:
 Path.cwd()


PosixPath('/net/trapnell/vol1/home/mdcolon/proj/morphseq/segmentation_sandbox')

## Block 1: Setup Test Data

First, we'll create some test data to work with.

In [ ]:
# BLOCK 1: Setup and Initialize Test Data
print("=== BLOCK 1: Setup Test Data ===")

# Create a test QC DataFrame with sample data
test_data = [
    {
        'experiment_id': '20250101',
        'video_id': '20250101_A01',
        'image_id': '20250101_A01_t001',
        'qc_flag': None,
        'notes': None,
        'annotator': None
    },
    {
        'experiment_id': '20250101',
        'video_id': '20250101_A01',
        'image_id': '20250101_A01_t002',
        'qc_flag': None,
        'notes': None,
        'annotator': None
    },
    {
        'experiment_id': '20250101',
        'video_id': '20250101_B01',
        'image_id': '20250101_B01_t001',
        'qc_flag': None,
        'notes': None,
        'annotator': None
    },
    {
        'experiment_id': '20250102',
        'video_id': '20250102_A01',
        'image_id': '20250102_A01_t001',
        'qc_flag': None,
        'notes': None,
        'annotator': None
    }
]

qc_df = pd.DataFrame(test_data)

# Save to test directory
test_dir = Path('test')
test_dir.mkdir(exist_ok=True)
qc_csv_path = test_dir / 'demo_qc.csv'
qc_df.to_csv(qc_csv_path, index=False)

print(f"Created test QC file with {len(qc_df)} images at: {qc_csv_path}")
print("Initial QC data:")
display(qc_df)

## Block 2: Manual QC Annotations

Now we'll demonstrate how to manually flag images with quality issues.

In [ ]:
# BLOCK 2: Manual QC Annotations
print("=== BLOCK 2: Manual QC Annotations ===")

# Load the current QC data
qc_df = pd.read_csv(qc_csv_path)

print("Available QC flags:")
for flag, description in QC_FLAGS.items():
    print(f"  {flag}: {description}")

# Example 1: Flag a single image as BLUR
print("\n1. Flagging image as BLUR...")
image_id = '20250101_A01_t001'
mask = qc_df['image_id'] == image_id
qc_df.loc[mask, 'qc_flag'] = 'BLUR'
qc_df.loc[mask, 'notes'] = 'Manual review - image appears blurry'
qc_df.loc[mask, 'annotator'] = 'mcolon'

# Example 2: Flag multiple images by video
print("2. Flagging frames in a video as DARK...")
image_id = '20250101_A01_t002'
mask = qc_df['image_id'] == image_id
qc_df.loc[mask, 'qc_flag'] = 'DARK'
qc_df.loc[mask, 'notes'] = 'Video frames too dark for analysis'
qc_df.loc[mask, 'annotator'] = 'mcolon'

# Save updated data
qc_df.to_csv(qc_csv_path, index=False)

print("\nUpdated QC data after manual annotations:")
flagged_data = qc_df.dropna(subset=['qc_flag'])
display(flagged_data)

## Block 3: Automatic QC Annotations

Here we simulate how automatic QC detection would work.

In [ ]:
# BLOCK 3: Automatic QC Annotations
print("=== BLOCK 3: Automatic QC Annotations ===")

qc_df = pd.read_csv(qc_csv_path)

# Simulate automatic blur detection
print("1. Simulating automatic blur detection...")
auto_blur_image = '20250101_B01_t001'
mask = qc_df['image_id'] == auto_blur_image
qc_df.loc[mask, 'qc_flag'] = 'BLUR'
qc_df.loc[mask, 'notes'] = 'Automatic: Laplacian variance < threshold (blur_threshold=100)'
qc_df.loc[mask, 'annotator'] = 'automatic'

# Simulate automatic brightness detection
print("2. Simulating automatic brightness detection...")
auto_dark_image = '20250102_A01_t001'
mask = qc_df['image_id'] == auto_dark_image
qc_df.loc[mask, 'qc_flag'] = 'DARK'
qc_df.loc[mask, 'notes'] = 'Automatic: Mean brightness < threshold (brightness_threshold=50)'
qc_df.loc[mask, 'annotator'] = 'automatic'

# Save updated data
qc_df.to_csv(qc_csv_path, index=False)

print("\nUpdated QC data after automatic annotations:")
auto_flagged = qc_df[qc_df['annotator'] == 'automatic']
display(auto_flagged)

## Block 4: Analyzing QC Data

Use the utility functions to analyze the QC data.

In [ ]:
# BLOCK 4: Analyzing QC Data
print("=== BLOCK 4: Analyzing QC Data ===")

qc_df = pd.read_csv(qc_csv_path)

# Get flagged images
flagged_images = get_flagged_images(qc_df)
print(f"Flagged images (excluding PASS): {len(flagged_images)}")
print(f"  {flagged_images}")

# Get unflagged images
unflagged_images = get_unflagged_images(qc_df)
print(f"\nUnflagged images: {len(unflagged_images)}")
print(f"  {unflagged_images}")

# Get images by specific flag
blur_images = get_images_by_flag(qc_df, 'BLUR')
print(f"\nImages flagged as BLUR: {len(blur_images)}")
print(f"  {blur_images}")

# Get images by annotator
manual_images = get_images_by_annotator(qc_df, 'mcolon')
auto_images = get_images_by_annotator(qc_df, 'automatic')
print(f"\nManually flagged images: {len(manual_images)}")
print(f"  {manual_images}")
print(f"\nAutomatically flagged images: {len(auto_images)}")
print(f"  {auto_images}")

# QC summary
print("\nQC Summary by flag:")
flag_counts = qc_df['qc_flag'].value_counts()
print(flag_counts)

print("\nQC Summary by annotator:")
annotator_counts = qc_df['annotator'].value_counts()
print(annotator_counts)

## Block 5: Removing and Updating QC Flags

Sometimes you need to remove or update existing QC flags.

In [ ]:
# BLOCK 5: Removing and Updating QC Flags
print("=== BLOCK 5: Removing and Updating QC Flags ===")

qc_df = pd.read_csv(qc_csv_path)

print("Current flagged images:")
flagged_before = qc_df.dropna(subset=['qc_flag'])
display(flagged_before[['image_id', 'qc_flag', 'annotator']])

# Remove a QC flag
print("\n1. Removing QC flag from one image...")
image_to_clear = '20250101_A01_t002'
mask = qc_df['image_id'] == image_to_clear
qc_df.loc[mask, 'qc_flag'] = None
qc_df.loc[mask, 'notes'] = None
qc_df.loc[mask, 'annotator'] = None

# Update an existing flag
print("2. Updating an existing QC flag...")
image_to_update = '20250101_A01_t001'
mask = qc_df['image_id'] == image_to_update
qc_df.loc[mask, 'qc_flag'] = 'OUT_OF_FOCUS'
qc_df.loc[mask, 'notes'] = 'Manual review - changed from BLUR to OUT_OF_FOCUS'
qc_df.loc[mask, 'annotator'] = 'mcolon'

# Save updated data
qc_df.to_csv(qc_csv_path, index=False)

print("\nUpdated flagged images:")
flagged_after = qc_df.dropna(subset=['qc_flag'])
display(flagged_after[['image_id', 'qc_flag', 'annotator']])

## Block 6: Final Summary

Review the final state of our QC data.

In [ ]:
# BLOCK 6: Final Summary and Cleanup
print("=== BLOCK 6: Final Summary ===")

qc_df = pd.read_csv(qc_csv_path)

print("Final QC data:")
display(qc_df)

print(f"\nTotal images: {len(qc_df)}")
print(f"Flagged images: {len(qc_df.dropna(subset=['qc_flag']))}")
print(f"Unflagged images: {len(qc_df[qc_df['qc_flag'].isna()])}")

print("\nFlag distribution:")
flag_dist = qc_df['qc_flag'].value_counts()
print(flag_dist)

print("\nAnnotator distribution:")
annotator_dist = qc_df['annotator'].value_counts()
print(annotator_dist)

print(f"\nDemo complete! Check the results in: {qc_csv_path}")

## Summary

This notebook demonstrated:

1. **Setup**: Creating initial QC data structure
2. **Manual QC**: Adding manual quality control flags with human annotators
3. **Automatic QC**: Simulating automatic detection and flagging
4. **Analysis**: Using utility functions to analyze QC data
5. **Updates**: Removing and updating existing QC flags
6. **Summary**: Reviewing the final state of QC data

### Key Functions Used:

- `get_flagged_images()`: Get list of flagged images
- `get_unflagged_images()`: Get list of unflagged images
- `get_images_by_flag()`: Filter images by specific QC flag
- `get_images_by_annotator()`: Filter images by annotator
- Direct DataFrame manipulation for adding/removing flags

### Next Steps:

- Integrate with actual image processing pipeline
- Add automatic blur detection algorithms
- Create visualization tools for QC data
- Set up batch processing workflows